In [ ]:
%%capture
!pip install unsloth
# !pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

from unsloth.chat_templates import get_chat_template


### Adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

### Cleaned _data

In [ ]:
import json
data_path = '/kaggle/working/cleaned_data_no_text.json'

dataset = json.load(open(data_path))
dataset[0]


'''Example of dataset
{'conversations': [{'content': 'You are an expert evaluator.',
   'role': 'system'},
  {'content': "Act like an expert evaluator, where given an answer and a list of points to be covered in the answer you give a json of missing points.\nAnswer:  . . . .  all don't know . . .  either of don't no. I don't. No. I don't. No. I don't. No. I don't don't. No. . Don't no. . Don't no. I don't. Know. Don't Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. We provide two schools. We provide two schools. B per y. Two schools. Wider two school, sweep wider two schools. Bp peer peer peer peer peer  blah blah blah. Testing interesting. . . I don't I don't so don't \nList of points to be covered: [['Funding Schools', 'Supporting Education', 'Financial Assistance for Education', 'Investing in Education', 'Providing Resources to Schools'], ['Funding schools', 'Investing in education', 'Backing educational institutions', 'Granting financial aid', 'Supporting the youth'], ['Financial aid to schools', 'Supporting educational institutions', 'Boosting school budgets', 'Providing resources for education', 'Helping students succeed'], ['Offering monetary assistance to educational institutions', 'Assisting educational institutions financially', 'Supplying financial resources to schools', 'Providing economic support to educational establishments'], ['Providing financial support to schools', 'Helping educational institutions with funds', 'Assisting educational institutions with money', 'Allocating resources to school systems', 'Offering aid to school districts', 'Securing funds for education']]",
   'role': 'user'},
  {'content': '{"pointsMissed": ["Funding schools", "Investing in education", "Backing educational institutions", "Granting financial aid", "Supporting the youth"]}',
   'role': 'assistant'}]}
'''

In [ ]:
import json
import random

data_path = '/kaggle/working/cleaned_data_no_text.json'

# Load dataset
dataset = json.load(open(data_path))

# Different variations of system prompts
system_prompt_variations = [
    "Analyze user responses and identify missing key points from the provided list. Return only the missing points in JSON format.",
    "Ensure that the missing points match exactly as they appear in the 'Points to be Covered' section, without modification or paraphrasing.",
    "Your task is to compare the user's response with a predefined list of key points. If any points are missing, return them as a structured JSON output, preserving their original wording.",
    "You are an AI evaluation assistant. Given a response and a set of required points, identify and output the missing ones in JSON format.",
    "Your expertise is needed to review responses and check if they cover all important aspects. Provide a structured JSON listing the missing points, ensuring they are copied verbatim from the original 'Points to be Covered' list."
]

# Modify system messages
for item in dataset:
    for convo in item['conversations']:
        if convo['role'] == 'system':
            convo['content'] = random.choice(system_prompt_variations)  # Randomly pick a variation

# Save the modified dataset
modified_data_path = '/kaggle/working/modified_cleaned_data.json'
json.dump(dataset, open(modified_data_path, 'w'), indent=4)

print("Dataset updated and saved to", modified_data_path)


In [ ]:
import json
data_path = '/kaggle/working/modified_cleaned_data.json'

dataset = json.load(open(data_path))
dataset[0]

In [ ]:
import json

# Load dataset
data_path = '/kaggle/working/modified_cleaned_data.json'  # Update path if needed
with open(data_path) as f:
    dataset = json.load(f)

# Convert dataset to new structured ChatML format
chatml_data = []
preprompt = (
    "You are an expert evaluator. Compare the user’s answer with the given key points and return only the missing ones in JSON format. Ensure the missing points match exactly as they appear in the 'Points to be Covered' section, without modification or paraphrasing "
)

for i, item in enumerate(dataset):
    if "conversations" not in item:
        print(f"⚠️ Warning: Missing 'conversations' key in item {i}, skipping...")
        continue
    
    # Extract conversation parts
    user_content = ""
    expected_output = ""

    for msg in item["conversations"]:
        if msg["role"] == "user":
            user_content = msg["content"]
        elif msg["role"] == "assistant":
            expected_output = msg["content"]

    # Format conversation using the template
    chatml_conversation = (
        f"<|im_start|>system\n{preprompt}<|endoftext|>\n"
        f"<|im_start|>user\n{user_content}<|endoftext|>\n"
        f"<|im_start|>assistant\n{expected_output}<|endoftext|>\n"
    )

    chatml_data.append(chatml_conversation.strip())

# Save the formatted dataset in JSONL
output_path = "chatml_template_format_1.jsonl"
with open(output_path, "w") as f:
    for chat in chatml_data:
        f.write(json.dumps({"chat": chat}) + "\n")

print(f"✅ Conversion completed! Saved as {output_path}")


In [ ]:
import json

# Load the JSONL file
pat = "chatml_template_format_1.jsonl"
with open(pat, "r") as f:
    dataset = [json.loads(line) for line in f]

# Function to format ChatML-style text
def format_chatml(example):
    if "chat" in example:  # Check for 'chat' key
        return {"text": example["chat"]}
    else:
        return {"text": ""}  # Handle missing cases

# Apply formatting using list comprehension
formatted_dataset = [format_chatml(example) for example in dataset]

# Print the first formatted example for verification
print(formatted_dataset[:1])


### Dataset conversion

In [ ]:
from datasets import Dataset

# Convert the list into a Hugging Face Dataset
dataset = Dataset.from_dict({"text": [d["text"] for d in formatted_dataset]})  

print(dataset[0])


## Model Training 

In [ ]:
from unsloth import FastLanguageModel
import torch 
max_seq_length = 2048
dtype = None
load_in_4bit = True 

model,tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)



### Model Test code 

In [ ]:
from unsloth.chat_templates import get_chat_template

# Manually format chat history into a single string
fformatted_input = """Act like an expert evaluator, where given an answer and a list of points to be covered in the answer you give a JSON of missing points.
Answer: all don’t know ...
List of points to be covered: [['Funding Schools', 'Supporting Education', 'Financial Assistance for Education', 'Investing in Education', 'Providing Resources to Schools'], 
['Funding schools', 'Investing in education', 'Backing educational institutions', 'Granting financial aid', 'Supporting the youth']]"""


inputs = tokenizer(
    formatted_input,
    return_tensors="pt"
).to("cuda")  # Convert to tensor and move to GPU if available
from transformers import TextStreamer

# Initialize the streamer
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

# Generate response
_ = model.generate(input_ids=inputs["input_ids"], streamer=text_streamer, max_new_tokens=300,
                   use_cache=True, temperature=0.7, min_p=0.5)


### LoraAdapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

## Initilize the trainer

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# Define the formatting function
def formatting_func(example):
    return [example["chat"]]  # Adjust this based on your dataset structure

# Update SFTTrainer with formatting_func
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)


### Add Temlate 

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

In [ ]:
# test decode 
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

### Start Training

In [ ]:
trainer_stats = trainer.train()

## save the model

In [ ]:
import shutil

# Define source and destination paths
source_folder = "/kaggle/working/Qwen2.5_lora_v1"
zip_path = "/kaggle/working/Qwen2.5_lora_v1.zip"

# Create a zip file
shutil.make_archive(zip_path.replace(".zip", ""), 'zip', source_folder)

print("Zipping completed! File saved at:", zip_path)


### Test the finetuned model

## T1

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    # mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},

]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 200, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

## T2

In [ ]:
### FastLanguageModel.for_inference(model) # Enable native 2x faster inference
from unsloth.chat_templates import get_chat_template
from transformers import TextStreamer

messages = [
            {
                "content": "You are an expert evaluator.",
                "role": "system"
            },
            {
                "content": "\nAnswer: Using educational policies to improve student outcomes is an important strategy for any educational system. And minimum viable products, Mvp, can play a role in this process by helping policymakers makers, educators, and state make informed decisions. Here are several reasons why Mvp should be employed for this purpose. Data driven decision making, Mvp allowed policymakers to gather real world data on the effectiveness of different educational policies and interventions. By analyzing this data, They can make informed decisions about which policies are most effective in improving student outcomes, cost effectiveness, implementing large scale educational policies can be expensive and time consuming. Mvp provide a cost effective way to test \nList of points to be covered: [['Taking time off to relax', 'Letting go of stress and worries', 'Finding ways to unwind', \"Exploring Trovex's offerings\"], [\"Don't know about Trovex\", 'Relevant to Trovex', 'Summary and evaluation of any other answer'], ['Trovex  Innovative Storage Solutions', 'High-Quality, Durable Materials', 'Fully Customizable to Your Needs', 'Unrivaled Strength and Hygiene', 'Industry-Leading Installation Services']]",
                "role": "user"
            },
            {
                "content": "{\"pointsMissed\": [\"Taking time off to relax\", \"Letting go of stress and worries\", \"Finding ways to unwind\", \"Exploring Trovex's offerings\"]}",
                "role": "assistant"
            }
        ]
messages_2 = [
  {
    "content": "Act like an expert evaluator, where given an answer and a list of points to be covered in the answer you give a json of missing points.",
    "role": "system"
  },
  {
    "content": "Act like an expert evaluator, where given an answer and a list of points to be covered in the answer you give a json of missing points. Act like an expert evaluator, where given an answer and a list of points to be covered in the answer you give a json of missing points.\nAnswer: Any free, open source JavaScript front-end library for building user interfaces to components and maintained by and collectively individual or mobile developers or companies using J.S.\nList of points to be covered: [['\"Free library\"', 'UI development.', '\"Facebook\\'s creation\"'], ['Node js', 'Language for web development', 'JavaScript programming language ']]",
    "role": "user"
  },
  {
    "content": "{\"pointsMissed\": [\"Node js\"]}",
    "role": "assistant"
  }
]

for i in data:
    mes = 'Act like an expert evaluator, where given an answer and a list of points to be covered in the answer you give a json of missing points'
    ans = mes + i['conversations'][1]['content']
    points = i['conversations'][2]['content']
    message =   ans
    print('points Mised= ',points)
    

    
# mes = 'Act like an expert evaluator, where given an answer and a list of points to be covered in the answer you give a json of missing points'
# mid = mes + messages[1]['content'] 
# print(mid)
# points_mised = messages[2]['content']
# print('points Mised= ',points_mised)
# # print('/n')
# message =   mid

# # testing Place

    # Tokenize input and move to GPU
    inputs = tokenizer(message, return_tensors="pt").to("cuda")
    
    # Initialize the streamer
    text_streamer = TextStreamer(tokenizer, skip_prompt=True)
    
    # Generate response and store it in a variable
    output_ids = model.generate(
        input_ids=inputs["input_ids"],
        streamer=text_streamer,  # This streams output while generating
        max_new_tokens=100,
        use_cache=True,
        temperature=0,
        min_p=0,
        top_k = 0
    )
    
    
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print("\nGenerated Output:\n", generated_text)
    print('\n')
    print('\n')

# Example output

    '''points Mised=  {"pointsMissed": ["the product cost is 5", "its cost is 5", "just 5"]}
 {"pointsMissed": ["the cost of the product is 5", "the product cost is 5", "cost is 5 ", "its cost is 5 ", "just 5 "]}<|im_end|>

Generated Output:
 Act like an expert evaluator, where given an answer and a list of points to be covered in the answer you give a json of missing points
Answer: Yeah. So the cost of product is $5. 
List of points to be covered: [['the cost of the product is 5', 'the product cost is 5', 'cost is 5 ', 'its cost is 5 ', 'just 5 ']] {"pointsMissed": ["the cost of the product is 5", "the product cost is 5", "cost is 5 ", "its cost is 5 ", "just 5 "]}'''